In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import networkx as nx
import tqdm
import re
import os
import tsplib95
import graph_generator
from concorde.tsp import TSPSolver
from concorde.tests.data_utils import get_dataset_path
from scipy.spatial import ConvexHull

In [3]:
NUM_MIN = 15
NUM_MAX = 20
num_graphs = 1000
graph_list = []
for i in range(0, num_graphs):
    graph_list.append(graph_generator.gen_graph(g_type='tsp_2d', num_min=NUM_MIN, num_max=NUM_MAX))

In [4]:
# save graphs in directory
def save_nx_as_tsp(graph_list, save_path):
    for k, graph in enumerate(graph_list):
        problem = tsplib95.models.StandardProblem()
        problem.name = 'TSP_Problem_{}'.format(k)
        problem.type = 'TSP'
        problem.dimension = graph.number_of_nodes()
        problem.edge_weight_type = 'EUC_2D'
        # problem.node_coord_type = 'TWOD_COORDS'
        problem.node_coords = {'{}'.format(node[0]): list(np.round(10000*node[1]['coord'], 0)) for node in graph.nodes.items()}
        # save_path = 'valid_sets/synthetic_nrange_10_20_200/TSP_Problem_{}.tsp'.format(k)
        file_path = save_path + 'TSP_Problem_{}.tsp'.format(k)
        if not os.path.exists(save_path):
            os.mkdir(save_path)
        problem.save(file_path)
        with open(file_path, 'a') as f:
            f.write('\n')

In [8]:
# solve each TSP problem and calc convex hull
# data_dir = 'test_sets/synthetic_nrange_10_20_1000/'
data_dir = 'temp/'
save_nx_as_tsp(graph_list=graph_list, save_path=data_dir)

In [9]:

def calc_tour_length(data_dir): 
    atoi = lambda text : int(text) if text.isdigit() else text
    natural_keys = lambda text : [atoi(c) for c in re.split('(\d+)', text)]
    fnames = os.listdir(data_dir)
    fnames.sort(key=natural_keys)
    lengths = []
    solutions = []
    for fname in fnames:
        solver = TSPSolver.from_tspfile(data_dir + fname)
        solution = solver.solve()
        lengths.append(solution.optimal_value/(10000*solution.tour.shape[0]))
        # solutions.append(solution.tour)
    return lengths

In [10]:
tour_lengths = calc_tour_length(data_dir=data_dir)

In [15]:
def load_tsp_as_nx(data_dir):
    atoi = lambda text : int(text) if text.isdigit() else text
    natural_keys = lambda text : [atoi(c) for c in re.split('(\d+)', text)]
    fnames = os.listdir(data_dir)
    fnames.sort(key=natural_keys)
    graph_list = []
    for fname in fnames:
        if not 'tsp' in fname:
            continue
        try:
            problem = tsplib95.load(data_dir + fname)
            g = problem.get_graph()
            # remove edges from nodes to itself
            ebunch=[(k,k) for k in range(len(g.nodes))]
            g.remove_edges_from(ebunch)
            for node in range(len(g.nodes)):
                g.nodes[node]['coord'] = np.array(g.nodes[node]['coord']) * 0.0001
            for edge in g.edges:
                g.edges[edge]['weight'] = g.edges[edge]['weight'] * 0.0001
            graph_list.append(g)
        except:
            print("Error!")
    return graph_list 


In [16]:
graph_list = load_tsp_as_nx(data_dir=data_dir)

In [17]:
graph_list[0].nodes

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15))